In [439]:
class Card:
    def __init__(self, rank, suit):
        self.rank = rank
        self.suit = suit

    def __repr__(self):
        suits = {
            "S": "♠",
            "H": "♥",
            "D": "♦",
            "C": "♣",
        }
        ranks = {
            14: "A",
            11: "J",
            12: "Q",
            13: "K",
        }
        if self.rank in ranks:
            rank = ranks[self.rank]
        else:
            rank = str(self.rank)
        return f"{rank}{suits[self.suit]}"

    def __eq__(self, other):
        return self.rank == other.rank and self.suit == other.suit

    def __lt__(self, other):
        return self.rank < other.rank

    def __hash__(self):
        return hash((self.rank, self.suit))

In [440]:
class CardDeck:
    def __init__(self):
        self.deck = [Card(rank, suit) for rank in range(2, 15) for suit in "SHDC"]
    def __repr__(self):
        return str(self.deck)
    def __len__(self):
        return len(self.deck)
    def __getitem__(self, position):
        return self.deck[position]
    def shuffleCards(self):
        import random
        random.shuffle(self.deck)
    def drawCards(self, n):
        return [self.deck.pop() for i in range(n)]
    def numCards(self):
        return len(self.deck)

In [441]:
class PokerHand:
    def __init__(self, cards):
        self.cards = cards
        self.cards.sort()
        self.cards.reverse()
        self.rank = self.getRank()
    def __repr__(self):
        return str(self.cards)
    def __len__(self):
        return len(self.cards)
    def __getitem__(self, position):
        return self.cards[position]
    def getRank(self):
        if self.isRoyalFlush():
            return 10
        elif self.isStraightFlush():
            return 9
        elif self.isFourOfAKind():
            return 8
        elif self.isFullHouse():
            return 7
        elif self.isFlush():
            return 6
        elif self.isStraight():
            return 5
        elif self.isThreeOfAKind():
            return 4
        elif self.isTwoPair():
            return 3
        elif self.isPair():
            return 2
        else:
            return 1.0 * self.cards[0].rank / 14
    def isRoyalFlush(self):
        if self.isStraightFlush() and self.cards[0].rank == 14:
            return True
        else:
            return False
    def isStraightFlush(self):
        if self.isFlush() and self.isStraight():
            return True
        else:
            return False
    def isFourOfAKind(self):
        if self.cards[0].rank == self.cards[3].rank or self.cards[1].rank == self.cards[4].rank:
            return True
        else:
            return False
    def isFullHouse(self):
        if self.isThreeOfAKind() and self.isPair():
            return True
        else:
            return False
    def isFlush(self):
        if self.cards[0].suit == self.cards[1].suit == self.cards[2].suit == self.cards[3].suit == self.cards[4].suit:
            return True
        else:
            return False
    def isStraight(self):
        if self.cards[0].rank == self.cards[1].rank + 1 == self.cards[2].rank + 2 == self.cards[3].rank + 3 == self.cards[4].rank + 4:
            return True
        else:
            return False
    def isThreeOfAKind(self):
        if self.cards[0].rank == self.cards[2].rank or self.cards[1].rank == self.cards[3].rank or self.cards[2].rank == self.cards[4].rank:
            return True
        else:
            return False
    def isTwoPair(self):
        if self.cards[0].rank == self.cards[1].rank and self.cards[2].rank == self.cards[3].rank:
            return True
        elif self.cards[0].rank == self.cards[1].rank and self.cards[3].rank == self.cards[4].rank:
            return True
        elif self.cards[1].rank == self.cards[2].rank and self.cards[3].rank == self.cards[4].rank:
            return True
        else:
            return False
    def isPair(self):
        if self.cards[0].rank == self.cards[1].rank or self.cards[1].rank == self.cards[2].rank or self.cards[2].rank == self.cards[3].rank or self.cards[3].rank == self.cards[4].rank:
            return True
        else:
            return False

In [442]:
from enum import Enum

# class syntax
class Action(Enum):
    FOLD = 1
    CALL = 2
    RAISE = 3
    CHECK = 4
    ALLIN = 5

In [443]:
len(Action)

5

In [444]:
class PokerGame:
    def __init__(self, players, smallBlind):
        self.deck = CardDeck()
        self.deck.shuffleCards()
        self.flop = []
        self.players = players
        self.smallBlind = smallBlind
        self.round = 0
        self.roundBet = 0
        self.pot = 0
        self.playersTurn = 0
        self.playersLeft = len(self.players)
        self.playersLeftToPlayInThisRound = len(self.players)
    
    def startRound(self):
        for player in self.players:
            player.resetRoundBet()
        self.roundBet = 0
        if self.round == 0:
            self.handleCards(2)
            self.makeInitialBets()
        elif self.round == 1:
            self.flopCards(3)
        elif self.round == 2:
            self.flopCards(1)
        elif self.round == 3:
            self.flopCards(1)
        self.playersLeftToPlayInThisRound = self.playersLeft
    
    def makeInitialBets(self):
        if self.players[0].getBudget() < self.smallBlind:
            self.playerTakeAction(self.players[0], Action.ALLIN)
        else:
            self.playerTakeAction(self.players[0], Action.RAISE, self.smallBlind)
        if self.players[1].getBudget() < self.smallBlind * 2:
            self.playerTakeAction(self.players[1], Action.ALLIN)
        else:
            self.playerTakeAction(self.players[1], Action.RAISE, self.smallBlind * 2)
    
    def finishRound(self):
        self.round += 1
        if self.isGameOver():
            self.getWinner()
    
    def getAmountToCall(self, player):
        return self.roundBet - player.roundBet
    
    def getPlayerActions(self, player): #all in fix needed
        if player.isFolded() == False:
            if player.isAllIn:
                return [Action.CHECK]
            if self.getAmountToCall(player) == 0:
                return [Action.FOLD, Action.RAISE, Action.ALLIN, Action.CHECK]
            elif player.getBudget() < self.getAmountToCall(player):
                return [Action.FOLD, Action.ALLIN]
            elif player.getBudget() == self.getAmountToCall(player):
                return [Action.FOLD, Action.CALL, Action.ALLIN]
            else:
                return [Action.FOLD, Action.CALL, Action.RAISE, Action.ALLIN]
        else:
            return [Action.FOLD]
    
    def playerTakeAction(self, player, action, amount = 0):
        if self.playersTurn != self.players.index(player):
            raise ValueError
        if action == Action.FOLD:
            self.playerFold(player)
        elif action == Action.CALL:
            self.playerCall(player)
        elif action == Action.RAISE:
            self.playerRaise(player, amount)
        elif action == Action.CHECK:
            self.playerCheck(player)
        elif action == Action.ALLIN:
            self.playerAllIn(player, player.getBudget())
    
    def getCurrentPlayer(self):
        return self.players[self.playersTurn]

    def roundEnded(self):
        if self.playersLeftToPlayInThisRound == 0:
            return True
        else:
            return False
    
    def nextPlayer(self):
        lastPlayer = self.playersTurn
        while True:
            self.playersTurn += 1
            if self.playersTurn == len(self.players):
                self.playersTurn = 0
            if self.players[self.playersTurn].isFolded() == False:
                break
            if self.playersTurn == lastPlayer:
                break
        # if self.playersTurn == lastPlayer:
        #     self.finishRound()
    
    def playerRaise(self, player, amount):
        try:
            player.bet(amount)
            self.pot += amount
            self.roundBet = player.roundBet
            self.playersLeftToPlayInThisRound = self.playersLeft - 1
            self.nextPlayer()
        except ValueError:
            print("Not enough money")
    
    def playerCall(self, player):
        try:
            amount = self.getAmountToCall(player)
            player.bet(amount)
            self.pot += amount
            self.playersLeftToPlayInThisRound -= 1
            self.nextPlayer()
        except ValueError:
            print("Not enough money")
    
    def playerCheck(self, player):
        try:
            if not player.isAllIn and self.getAmountToCall(player) != 0:
                raise ValueError
            self.playersLeftToPlayInThisRound -= 1
            self.nextPlayer()
        except ValueError:
            print("Not enough money")
    
    def playerAllIn(self, player, amount):
        try:
            player.bet(amount)
            self.pot += amount
            self.roundBet = max(player.roundBet, self.roundBet)
            self.playersLeftToPlayInThisRound = self.playersLeft - 1
            self.nextPlayer()
        except ValueError:
            print("Not enough money")
    
    def playerFold(self, player):
        player.fold()
        self.playersLeftToPlayInThisRound -= 1
        self.playersLeft -= 1
        if self.playersLeft != 1:
            self.nextPlayer()
    
    def getWinner(self): #update this
        nonFoldedPlayers = [player for player in self.players if player.isFolded() == False]
        winner = nonFoldedPlayers[0]
        if len(nonFoldedPlayers) == 1:
            winner.budget += self.pot
            print(winner)
            return winner
        winnerBestHand = self.getBestHand(winner)
        for player in nonFoldedPlayers:
            playerBestHand = self.getBestHand(player)
            if playerBestHand.rank > winnerBestHand.rank:
                winner = player
                winnerBestHand = playerBestHand
        winner.budget += self.pot
        return winner
    
    def getBestHand(self, player):
        import itertools
        bestHand = PokerHand(self.flop)
        for comb in itertools.combinations(player.hand + self.flop, 5):
            comb = list(comb)
            if PokerHand(comb).rank > bestHand.rank:
                bestHand = PokerHand(comb)
        return bestHand
    
    def isGameOver(self):
        if self.round == 4 or self.playersLeft == 1:
            return True
        else:
            return False
    
    def handleCards(self, n):
        for player in self.players:
            player.hand = self.deck.drawCards(n)
    
    def flopCards(self, n):
        self.flop = self.flop + self.deck.drawCards(n)
    
    def getFlop(self):
        return self.flop
    
    def getPot(self):
        return self.pot

In [445]:
import random
class Poker:
    def __init__(self, players, smallBlind, buyin):
        self.players = players
        self.smallBlind = smallBlind
        self.buyin = buyin
        for player in self.players:
            player.budget = self.buyin
    
    def playUntilWinner(self):
        while self.areAllGamesOver() == False:
            print("New game started!")
            self.playRound()
            self.removePlayersWithNoMoney()
            self.resetPlayers()
            self.shiftPlayers()
            print("One game ended!")
        print("Game over! " + str(self.players[0].id) + " won!")

    def resetPlayers(self):
        for player in self.players:
            player.resetValues()

    def shiftPlayers(self):
        self.players.append(self.players.pop(0))
    
    def playRound(self):
        pokerGame = PokerGame(self.players, self.smallBlind)
        while pokerGame.isGameOver() == False:
            print("###################")
            print("New round started! " + str(pokerGame.round))
            pokerGame.startRound()
            while pokerGame.isGameOver() == False and pokerGame.roundEnded() == False:
                player = pokerGame.getCurrentPlayer()
                print("-------------------")
                print("player.id", player.id, "budget", player.budget)
                print(pokerGame.getFlop())
                print(pokerGame.getPot())
                print(player)
                actions = pokerGame.getPlayerActions(player)
                print("actions", actions)
                rand = random.randint(0, len(actions) - 1)
                action = actions[rand]
                print("action", action)
                if(action == Action.RAISE):
                    raiseAmount = random.randint(pokerGame.getAmountToCall(player), player.budget)
                    pokerGame.playerTakeAction(player, action, raiseAmount)
                else:
                    pokerGame.playerTakeAction(player, action)
                print("-------------------")
            print(pokerGame.pot)
            pokerGame.finishRound()
        print("Round ended! " + str(pokerGame.round))
        print("###################")
        for player in self.players:
            print("player with id:" , player.id, "has", player.budget, player.hand)

    def removePlayersWithNoMoney(self):
        players = []
        for player in self.players:
            if player.budget > 0:
                players.append(player)
        self.players = players
        print(self.players)
    
    def areAllGamesOver(self):
        return len(self.players) == 1

In [446]:
class PokerPlayer:
    def __init__(self, id, budget=0):
        self.id = id
        self.budget = budget
        self.bettedAmount = 0
        self.roundBet = 0
        self.hand = []
        self.folded = False
        self.isAllIn = False
    def getBudget(self):
        return self.budget
    def getBettedAmount(self):
        return self.bettedAmount
    def bet(self, amount):
        if(amount > self.budget):
            raise ValueError("You don't have enough money to bet that much!")
        self.roundBet += amount
        self.budget -= amount
        self.bettedAmount += amount
        if self.budget == 0:
            self.isAllIn = True
    def resetValues(self):
        self.bettedAmount = 0
        self.roundBet = 0
        self.hand = []
        self.folded = False
        self.isAllIn = False
    def resetRoundBet(self):
        self.roundBet = 0
    def isFolded(self):
        return self.folded
    def fold(self):
        self.folded = True
    def __repr__(self):
        return str(self.hand)
    def __len__(self):
        return len(self.hand)
    def __getitem__(self, position):
        return self.hand[position]


In [447]:
player1 = PokerPlayer(1)
player2 = PokerPlayer(2)
player3 = PokerPlayer(3)
player4 = PokerPlayer(4)
player5 = PokerPlayer(5)

poker = Poker([player1, player2, player3, player4, player5], 10, 100)
poker.playUntilWinner()

New game started!
###################
New round started! 0
-------------------
player.id 3 budget 100
[]
30
[6♠, 10♦]
actions [<Action.FOLD: 1>, <Action.CALL: 2>, <Action.RAISE: 3>, <Action.ALLIN: 5>]
action Action.CALL
-------------------
-------------------
player.id 4 budget 100
[]
50
[5♠, A♦]
actions [<Action.FOLD: 1>, <Action.CALL: 2>, <Action.RAISE: 3>, <Action.ALLIN: 5>]
action Action.CALL
-------------------
-------------------
player.id 5 budget 100
[]
70
[K♥, A♣]
actions [<Action.FOLD: 1>, <Action.CALL: 2>, <Action.RAISE: 3>, <Action.ALLIN: 5>]
action Action.RAISE
-------------------
-------------------
player.id 1 budget 90
[]
117
[Q♦, 2♦]
actions [<Action.FOLD: 1>, <Action.CALL: 2>, <Action.RAISE: 3>, <Action.ALLIN: 5>]
action Action.RAISE
-------------------
-------------------
player.id 2 budget 80
[]
159
[5♣, 9♣]
actions [<Action.FOLD: 1>, <Action.CALL: 2>, <Action.RAISE: 3>, <Action.ALLIN: 5>]
action Action.FOLD
-------------------
-------------------
player.id 3 budget

In [448]:
# player1 = PokerPlayer(11, 1000)
# player2 = PokerPlayer(12, 1000)
# player3 = PokerPlayer(13, 1000)
# player4 = PokerPlayer(14, 1000)
# player5 = PokerPlayer(15, 1000)

# poker = PokerGame([player1, player2, player3, player4, player5], 10)

# while poker.isGameOver() == False:
#     print("New round started! " + str(poker.round))
#     poker.startRound()
#     while poker.roundEnded() == False:
#         player = poker.getCurrentPlayer()
#         print(poker.getFlop())
#         print(poker.getPot())
#         print(poker.getCurrentPlayer())
#         print(poker.getPlayerActions(poker.getCurrentPlayer()))
#         action = input("Action: ")
#         if action == "fold":
#             poker.playerTakeAction(player, Action.FOLD)
#         elif action == "call":
#             poker.playerTakeAction(player, Action.CALL)
#         elif action == "raise":
#             amount = int(input("Amount: "))
#             poker.playerTakeAction(player, Action.RAISE, amount)
#         elif action == "check":
#             poker.playerTakeAction(player, Action.CHECK)
#         elif action == "allin":
#             poker.playerTakeAction(player, Action.ALLIN)
#         else:
#             print("Invalid action")
#     poker.finishRound()
#     print("Round ended! " + str(poker.round))

# print("Game over!")
# print("The flop was: " + str(poker.flop))
# print("Player budgets:")
# for player in poker.players:
#     print("player with id:" , player.id, "has", player.budget, player.hand, "hand rank:", poker.getBestHand(player).rank)

In [449]:
cardDeck = CardDeck()
cardDeck.shuffleCards()
print(cardDeck)

[8♣, 8♦, 7♣, 9♥, 6♠, 10♦, 3♣, 9♦, Q♠, J♥, K♠, 2♠, 6♣, Q♥, 5♦, 4♥, 4♣, 5♠, 4♦, 7♦, 9♣, 4♠, J♦, 2♣, 8♠, A♥, 5♥, 10♣, 10♥, 6♦, 7♥, 5♣, K♣, A♦, K♦, Q♦, A♣, A♠, J♣, 9♠, 2♦, 2♥, 10♠, 3♠, 3♥, 7♠, Q♣, 8♥, 3♦, J♠, 6♥, K♥]


In [450]:
card = Card(1, "S")
card2 = Card(1, "H")
print(card == card2)

False


In [451]:
import itertools
flop = [Card(2, "S"), Card(3, "S"), Card(4, "S"), Card(5, "S"), Card(6, "S")]
hand = [Card(5, "H"), Card(6, "H")]
bestHand = PokerHand(flop)
allHands = []
for comb in itertools.combinations(hand + flop, 5):
    comb = list(comb)
    allHands.append(PokerHand(comb))
    if PokerHand(comb).rank > bestHand.rank:
        bestHand = PokerHand(comb)
print(bestHand)
print(len(allHands))

[6♠, 5♠, 4♠, 3♠, 2♠]
21
